In [1]:
##################################
#
# Implementation of linear logic recurrent neural network
#
# The architecture is a modified RNN, see the paper "Linear logic and recurrent neural networks".
# Our inputs are sequences of symbols taken from an alphabet of size num_classes. The length
# of the sequences is N. Our outputs are also sequences of length N from the same alphabet.
#
# Here "symbol" means a one hot vector.

# The next three lines are recommend by TF
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import collections
import six
import math
import time

from tensorflow.python.ops.rnn_cell_impl import _RNNCell as RNNCell
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops.math_ops import sigmoid
from tensorflow.python.ops.math_ops import tanh

# Our libraries
import ntm
import seqhelper
import learnfuncs

###########
# TODO/BUGS
#
# - Running on N = 10 crashes

In [2]:
##############
# GLOBAL FLAGS

use_model             = 'ntm' # ntm, pattern_ntm
task                  = 'copy' # copy, repeat copy, pattern
epoch                 = 200 # number of training epochs
num_classes           = 2 # number of symbols in the alphabet
N                     = 20 # length of input sequences
training_percent      = 0.01 # percentage used for training
batch_size            = 500 # take a smaller batch size (500 works) on Tesla
controller_state_size = 100 # dimension of the internal state space of the controller
memory_address_size   = 20 # number of memory locations
memory_content_size   = 5 # size of vector stored at a memory location
powers_ring1          = [-1,0,1] # powers available to pattern NTM

In [ ]:
#######################
# PREPARE TRAINING DATA
#
# Our sequences are of one-hot vectors, which we interpret as follows:
#
# [1.0, 0.0, 0.0] = 0
# [0.0, 1.0, 0.0] = 1
# [0.0, 0.0, 1.0] = 2 etc
#
# We write our sequences and functions referring to sequences of integers,
# and then convert to one-hot vectors for integration with TF.

###########
# COPY TASK
if( task == 'copy' ):
    func_to_learn = learnfuncs.f_identity
    N_out = N

##################
# REPEAT COPY TASK
# put n zeros before the 1, for a copy task with n + 1 copies
if( task == 'repeat copy' ):
    pattern = [0,1]
    func_to_learn = lambda s: learnfuncs.f_repetitionpattern(s,pattern)
    N_out = 2 * N

##############
# PATTERN TASK
if( task == 'pattern' ):
    pattern = [1,0,0,2,0]
    func_to_learn = lambda s: learnfuncs.f_repetitionpattern(s,pattern)
    N_out = 2 * N

seq_input = seqhelper.shuffled_seqs(N,num_classes)
one_hots = seqhelper.one_hot_vectors(num_classes)

seq_input_onehot = []
for i in seq_input:
    temp_list = []
    for j in i:
        temp_list.append(one_hots[j])
    seq_input_onehot.append(np.array(temp_list))

# Training output
seq_output = []

for i in seq_input:
    seq_output.append(func_to_learn(i))

seq_output_onehot = []
for i in seq_output:
    temp_list = []
    for j in i:
        temp_list.append(one_hots[j])
    seq_output_onehot.append(np.array(temp_list))

NUM_EXAMPLES = int(training_percent * len(seq_input))

test_input = seq_input_onehot[NUM_EXAMPLES:3*NUM_EXAMPLES]
test_output = seq_output_onehot[NUM_EXAMPLES:3*NUM_EXAMPLES]
train_input = seq_input_onehot[:NUM_EXAMPLES]
train_output = seq_output_onehot[:NUM_EXAMPLES]

print("Number of training examples: " + str(NUM_EXAMPLES) + " out of " + str(len(seq_input)) + " sequences.")
print("")
print("Under the chosen function, the sequence")
print(seq_input[0])
print("which is encoded as")
print(seq_input_onehot[0])
print("is mapped to")
print(seq_output[0])
print("which is encoded as")
print(seq_output_onehot[0])

#print("")
#print("The first one-hot encoded digit of the first three output sequences")
#print(test_output[0][0])
#print(test_output[1][0])
#print(test_output[2][0])

KeyboardInterrupt: 

In [ ]:
################
# DEFINE MODEL #
################

input_size = num_classes # dimension of the input space I

# inputs, we create N of them, each of shape [None,input_size], one for
# each position in the sequence
inputs = [tf.placeholder(tf.float32, [None,input_size]) for _ in range(N)]
targets = [tf.placeholder(tf.float32, [None,input_size]) for _ in range(N_out)]

# state_size is the number of hidden neurons in each layer
state_size = 0

if( use_model == 'ntm' ):
    state_size = controller_state_size + 2*memory_address_size + memory_address_size * memory_content_size
    cell = ntm.NTM(state_size,input_size,controller_state_size,memory_address_size,memory_content_size, [-1,0,1])
elif( use_model == 'pattern_ntm' ):
    state_size = controller_state_size + 4*memory_address_size + \
                memory_address_size * memory_content_size + \
                memory_address_size * len(powers_ring1)

    cell = ntm.PatternNTM(state_size,input_size,controller_state_size,
                          memory_address_size,memory_content_size, powers_ring1, [-1,0,1])

state = cell.zero_state(batch_size, tf.float32)

reuse = False

for i in range(N):
    output, state = cell(inputs[i],state,'NTM',reuse)
    reuse = True

# We only start recording the outputs of the controller once we have
# finished feeding in the input. We feed zeros as input in the second phase.
rnn_outputs = []
for i in range(N_out):
    output, state = cell(tf.zeros([batch_size,input_size]),state,'NTM',reuse)
    rnn_outputs.append(output)

# Final fully connected layer
E = tf.Variable(tf.truncated_normal([controller_state_size,input_size]))
F = tf.Variable(tf.constant(0.1, shape=[input_size]))

# prediction is a length N list of tensors of shape [None,input_size], where
# the jth row of prediction[d] is, for the jth input sequence in the batch,
# the probability distribution over symbols for the output symbol in position d.
logits = [tf.matmul(rnn_output, E) + F for rnn_output in rnn_outputs]
prediction = [tf.nn.softmax(logit) for logit in logits] 
ce = [tf.reduce_sum(targets[i] * tf.log(prediction[i])) for i in range(N_out)]

cross_entropy = -tf.add_n(ce)
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

mistakes = [tf.not_equal(tf.argmax(targets[i], 1), tf.argmax(prediction[i], 1)) for i in range(N_out)]
errors = [tf.reduce_mean(tf.cast(m, tf.float32)) for m in mistakes]

[<tf.Tensor 'gradients/NTM_29/split_grad/concat:0' shape=(500, 240) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_28/split_grad/concat:0' shape=(500, 240) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_27/split_grad/concat:0' shape=(500, 240) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_26/split_grad/concat:0' shape=(500, 240) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_25/split_grad/concat:0' shape=(500, 240) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_24/split_grad/concat:0' shape=(500, 240) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_23/split_grad/concat:0' shape=(500, 240) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_22/split_grad/concat:0' shape=(500, 240) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_21/split_grad/concat:0' shape=(500, 240) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_20/split_grad/concat:0' shape=(500, 240) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_19/split_grad

In [ ]:
# Initialise the model
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [ ]:
# Display the errors before training
feed_dict = {}
test_input_batch = test_input[:batch_size]
test_output_batch = test_output[:batch_size]
for d in range(N):
    in_node = inputs[d]
    ti = []
    for k in range(len(test_input_batch)):
        ti.append(test_input_batch[k][d]) # A vector giving the one-hot encoding of the dth symbol in the kth sequence
    feed_dict[in_node] = np.array(ti)
    
for d in range(N_out):
    out_node = targets[d]
    to = []
    for k in range(len(test_output_batch)):
        to.append(test_output_batch[k][d])
    feed_dict[out_node] = np.array(to)

# The first three digits of this should match the printout for the
# first three test output sequences given earlier
#print(sess.run(tf.argmax(targets[0],1),feed_dict))
#print(sess.run(tf.argmax(prediction[0],1),feed_dict))
#print(sess.run(tf.not_equal(tf.argmax(targets[0], 1), tf.argmax(prediction[0], 1)),feed_dict))

#print("")
#print("The mean of the errors in each digit for the test set:")
#incorrects = sess.run(errors, feed_dict)
#print(incorrects)
#print("Mean: " + str(np.mean(incorrects)))

In [ ]:
############
# TRAINING #
############

pre_train_time = time.time()

# Training
no_of_batches = int(len(train_input)/batch_size)
#print("Number of batches: " + str(no_of_batches))

# An annoying thing here is that we cannot use a list as a key in a 
# dictionary. The workaround we found on StackOverflow here:
# http://stackoverflow.com/questions/33684657/issue-feeding-a-list-into-feed-dict-in-tensorflow)

# epoch is a global var
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp = train_input[ptr:ptr+batch_size]
        out = train_output[ptr:ptr+batch_size]
        
        ptr += batch_size
        
        feed_dict = {}
        for d in range(N):
            in_node = inputs[d]
            # inp has dimensions [batch_size, N, num_classes] and we want to extract
            # the 2D Tensor of shape [batch_size, num_classes] obtained by setting the
            # second coordinate to d
            ti = []
            for k in range(batch_size):
                ti.append(inp[k][d])
            feed_dict[in_node] = np.array(ti)

        for d in range(N_out):
            out_node = targets[d]
            to = []
            for k in range(batch_size):
                to.append(out[k][d])
            feed_dict[out_node] = np.array(to)
            
        sess.run(minimize, feed_dict)
    current_mean = np.mean(sess.run(errors, feed_dict))
    print("Epoch - " + str(i+1) + ", Mean error of final batch in epoch - " + str(current_mean))

print("")
print("It took", time.time() - pre_train_time, "seconds to train.")

In [ ]:
###########
# TESTING #
###########

no_of_batches = int(len(test_input)/batch_size)
#print("Number of batches: " + str(no_of_batches))

error_means = []
ptr = 0
for j in range(no_of_batches):
    inp = test_input[ptr:ptr+batch_size]
    out = test_output[ptr:ptr+batch_size]
    ptr += batch_size
        
    feed_dict = {}
    for d in range(N):
        in_node = inputs[d]
        ti = []
        for k in range(batch_size):
            ti.append(inp[k][d])
        feed_dict[in_node] = np.array(ti)

    for d in range(N_out):
        out_node = targets[d]
        to = []
        for k in range(batch_size):
            to.append(out[k][d])
        feed_dict[out_node] = np.array(to)
            
    current_mean = np.mean(sess.run(errors, feed_dict))
    error_means.append(current_mean)
    print("Batch - " + str(j+1) + ", Mean error - " + str(current_mean))

final_error = np.mean(error_means)

# The first three digits of this should match the printout for the
# first three test output sequences given earlier
#data = sess.run([tf.argmax(targets[0],1), tf.argmax(prediction[0],1)],feed_dict)

#print("First digits of test outputs (actual)")
#print(data[0])
#print("First digits of test outputs (predicted)")
#print(data[1])

# print the mean of the errors in each digit for the test set.
#incorrects = sess.run(errors, feed_dict)
# print(incorrects)

print("")        
print("###########")
print("# Summary #")
print("###########")
print("")
print("model         - " + use_model)
print("task name     - " + task)
print("# epochs      - " + str(epoch))
print("ring powers   - " + str(powers_ring1))
print("train percent - " + str(training_percent))
print("(css,mas,mcs) - (" + str(controller_state_size) + "," + str(memory_address_size) + "," + str(memory_content_size) + ")")
print("# weights     - " + str(ntm.count_number_trainable_params()))
print("N             - " + str(N))
print("N_out         - " + str(N_out))
print("num_classes   - " + str(num_classes))
print("")
print("error         - " + str(final_error))
sess.close()